In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import time
from datetime import datetime

import gmaps
import requests
# Import the API keys.
from config import weather_api_key
from config import g_key

# Create new Latitudes and longitudes and find nearest city

In [2]:
#3 Create a set of 2000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
# zip will combine two lists into a tuple
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
#4 Get the nearest city
# Create a list for holding the cities
cities=[]

# Find the nearest city and add to the list
for coordinate in coordinates:
    lat=coordinate[0]
    lng=coordinate[1]
    city=citipy.nearest_city(lat,lng).city_name
    if city not in cities:
        cities.append(city)
# Print the city count as a check
print({len(coordinates),len(cities)})

{2000, 774}


# Get Weather data for cities

In [4]:
#5 Perform API call with OpenWeatherMap
#6 Retrieve info

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        # Get the description for the weather
        city_desc = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Description":city_desc,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=70595b3083ba566a9bcc8d66a3ec5952
Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | jamestown
Processing Record 2 of Set 1 | bengkulu
Processing Record 3 of Set 1 | san carlos de bariloche
Processing Record 4 of Set 1 | sari
Processing Record 5 of Set 1 | biscarrosse
Processing Record 6 of Set 1 | osypenko
Processing Record 7 of Set 1 | nikolskoye
Processing Record 8 of Set 1 | bredasdorp
Processing Record 9 of Set 1 | saleaula
City not found. Skipping...
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | khatanga
Processing Record 12 of Set 1 | kita
Processing Record 13 of Set 1 | punta arenas
Processing Record 14 of Set 1 | mataura
Processing Record 15 of Set 1 | albany
Processing Record 16 of Set 1 | upernavik
Processing Record 17 of Set 1 | svetlaya
Processing Record 18 of Set 1 | provideniya
Processing Record 19 of Set 1 | cherskiy
Processing Record 20 o

In [6]:
# Store in data frame and print to file
city_data_df=pd.DataFrame(city_data)
# Create the output file (CSV).
output_data_file = "cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.0970,-79.2353,36.72,83,1,4.00,US,2021-12-05 02:38:00
1,Bengkulu,-3.8004,102.2655,77.85,84,100,6.82,ID,2021-12-05 02:42:24
2,San Carlos De Bariloche,-41.1456,-71.3082,55.15,58,0,3.44,AR,2021-12-05 02:34:19
3,Sari,36.5633,53.0601,46.58,47,0,4.23,IR,2021-12-05 02:42:24
4,Biscarrosse,44.3945,-1.1672,48.15,81,75,14.97,FR,2021-12-05 02:42:25
5,Osypenko,46.9136,36.8253,35.83,92,100,13.40,UA,2021-12-05 02:42:25
6,Nikolskoye,59.7035,30.7861,10.02,93,91,6.22,RU,2021-12-05 02:38:53
7,Bredasdorp,-34.5322,20.0403,60.28,95,100,17.83,ZA,2021-12-05 02:38:54
8,Rikitea,-23.1203,-134.9692,76.17,72,89,8.79,PF,2021-12-05 02:41:28
9,Khatanga,71.9667,102.5000,-23.17,96,100,11.70,RU,2021-12-05 02:38:46
